In [2]:
from utils.read_data import get_train_targets, load_data, get_test_data, prepare_submission
from utils.generate_run_name import generate_run_name
import numpy as np
import logging
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import catboost as cb
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from autogluon.tabular import TabularPredictor
import logging
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin

c:\Users\Taheera Ahmed\code\tdt4173-project\tdt4173-project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
drop_cols = ['time', 'elevation:m', 'fresh_snow_1h:cm', 'ceiling_height_agl:m', 'snow_density:kgm3', 
             'wind_speed_w_1000hPa:ms', 'snow_drift:idx', 'fresh_snow_3h:cm', 'is_in_shadow:idx', 'dew_or_rime:idx', 'fresh_snow_6h:cm', 'prob_rime:p'] # this second line is columns with feature importance == 0

data_a, data_b, data_c = load_data(mean=True, remove_out=True, roll_avg=True, cust_feat=True, drop_cols=drop_cols, cycle_encoding = True)
X_test_a, X_test_b, X_test_c = get_test_data(mean=True, roll_avg=True, cust_feat=True, cycle_encoding = True)

# Assuming get_train_targets is defined and returns the features and target
X_train_a, y_a = get_train_targets(data_a)
X_train_b, y_b = get_train_targets(data_b)
X_train_c, y_c = get_train_targets(data_c)

In [5]:
class AutoGluonWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, path):
        self.path = path
        self.model = TabularPredictor.load(path)

    def fit(self, X, y):
        # AutoGluon's fit method is called during the initial training, 
        # so we don't need to call it here again.
        # Just return self to adhere to scikit-learn's fit method requirements.
        return self

    def predict(self, X):
        return self.model.predict(X)

In [8]:
# Now you can add your AutoGluon model to the base models
autogluon_model_a = AutoGluonWrapper("E:\\Jert Mira-A")

autogluon_predictions_a = autogluon_model_a.predict(X_train_a)
autogluon_predictions_test_a = autogluon_model_a.predict(X_test_a)

X_test_a = pd.concat([X_test_a, autogluon_predictions_test_a], axis=1)
X_train_a = pd.concat([X_train_a, autogluon_predictions_a], axis=1)

"""
cat_boost1: bayesian_search on params given feat_eng from 07.nov
cat_boost2: from best kaggle submission
"""
base_model_a = [
    ('autogluon', autogluon_model_a)
    ('cat_boost3', cb.CatBoostRegressor(random_state=42, silent=True)),
]
# Define meta-learners for each location
meta_learner_a = LinearRegression()

    # Create the stacking regressor for the current location
stacked_model = StackingRegressor(estimators=base_model_a, final_estimator=meta_learner_a)

# Create the whole model pipeline for the current location
whole_model_pipeline = Pipeline([
    ('stacked_model', stacked_model)
])

<>:15: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:15: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?

############################## WARNING ##############################
	Predictor Version: 0.8.2
	Current Version:   0.5.3
############################## WARNING ##############################



AssertionError: Predictor was created on version 0.8.2 but is being loaded with version 0.5.3. Please ensure the versions match to avoid instability. While it is NOT recommended, this error can be bypassed by specifying `require_version_match=False`.

In [ ]:
whole_model_pipeline.fit(X_train_a, y_a)
predictions = whole_model_pipeline.predict(X_test_a.drop(columns=["id", "prediction", "location"]))

# Prepare submission using the predictions
prepare_submission(X_test_a, predictions, run_name=run_name)